In [27]:
import os
current_path = os.getcwd()
print(current_path)

from google.colab import drive
drive.mount('/content/drive')

/root
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [30]:
import zipfile
DATA_IN_PATH = '/content/drive/MyDrive/Colab Notebooks/Robot융합/DeepLearning_NLP/data_in/'
file_list = ['labeledTrainData.tsv.zip', 'unlabeledTrainData.tsv.zip', 'testData.tsv.zip']

for file in file_list:
  zipRef = zipfile.ZipFile(DATA_IN_PATH + file, 'r')
  zipRef.extractall(DATA_IN_PATH)
  zipRef.close()

In [31]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

print("File size: ")

for file in os.listdir(DATA_IN_PATH) :
  if 'tsv' in file and 'zip' not in file:
    print(file.ljust(30) + str(round(os.path.getsize(DATA_IN_PATH + file)/1000000, 2)) + 'MB')

train_data = pd.read_csv(DATA_IN_PATH + 'labeledTrainData.tsv', header = 0, delimiter = '\t', 
                         quoting = 3)
print('The # of entire training data: {}'.format(len(train_data)))

train_length = train_data['review'].apply(len)
train_length.head()

File size: 
labeledTrainData.tsv          33.56MB
unlabeledTrainData.tsv        67.28MB
testData.tsv                  32.72MB
The # of entire training data: 25000


0    2304
1     948
2    2451
3    2247
4    2233
Name: review, dtype: int64

In [33]:
import re
import json
import pandas as pd
import numpy as np
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

DATA_IN_PATH = '/content/drive/MyDrive/Colab Notebooks/Robot융합/DeepLearning_NLP/data_in/'
train_data = pd.read_csv(DATA_IN_PATH + 'labeledTrainData.tsv', header = 0, delimiter = '\t',
                         quoting = 3)

In [42]:
review = train_data['review'][0] #첫번째 리뷰 가져와
review_text = BeautifulSoup(review, "html5lib").get_text() #html태그 제거
review_text = re.sub("[^a`zA-Z]", " ", review_text) #영어문자 제외한 나머진 전부 공백으로

nltk.download('stopwords')
stop_words = set(stopwords.words('english')) #영어 불용어 집합 구성

review_text = review_text.lower()
words = review_text.split() # 소문자 변환 후, 단어마다 나눠 리스트로 만듬
words = [w for w in words if not w in stop_words] #불용어 제외 리스트 구성
print(words)
clean_reivew = ' '.join(words) #단어리스트를 다시 하나의 글로 합침
print(clean_reivew)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['w', 'mj', 'w', 'z', 'mj', 'v', 'ja', 'mj', 'mj', 'mj', 'c', 'j', 'p', 'w', 'mj', 'b', 'mj', 'na', 'j', 'p', 'mj', 'l', 'mj', 'ba', 'b', 'mj', 'mj', 'ja', 'w', 'h']
w mj w z mj v ja mj mj mj c j p w mj b mj na j p mj l mj ba b mj mj ja w h


In [45]:
def preprocessing(reivew, remove_stopwords = False) :
  review_text = BeautifulSoup(review, "html5lib").get_text()
  review_text = re.sub("[^a-zA-Z]", " ", review_text)
  words = review_text.lower().split()

  if remove_stopwords:
    stops = set(stopwords.words("english"))
    words = [w for w in words if not w in stops]
    clean_review = ' '.join(words)
  else : #불용어 제거 안 하는 경우
    clean_review = ' '.join(words)
  return clean_review

clean_train_reviews = []
for review in train_data['review']:
  clean_train_reviews.append(preprocessing(review, remove_stopwords = True))

clean_train_df = pd.DataFrame({'review' : clean_train_reviews,
                               'sentiment' : train_data['sentiment']})

In [56]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_reviews)
text_sequences = tokenizer.texts_to_sequences(clean_train_reviews)

MAX_SEQUENCE_LENGTH = 174
train_inputs = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding = 'post')
print('Train data: ', train_inputs.shape)

train_labels = np.array(train_data['sentiment'])
print('Label: ', train_labels.shape)

Train data:  (25000, 174)
Label:  (25000,)


In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

DATA_IN_PATH = '/content/drive/MyDrive/Colab Notebooks/Robot융합/DeepLearning_NLP/data_in/'
DATA_OUT_PATH = '/content/drive/MyDrive/Colab Notebooks/Robot융합/DeepLearning_NLP/data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'
RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [58]:
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer = "char", sublinear_tf = True, 
                             ngram_range = (1,3), max_features = 5000)
X= vectorizer.fit_transform(reviews)
y= np.array(sentiment)

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=TEST_SPLIT,
                                                    random_state = RANDOM_SEED)
X_train.shape, y_train.shape, X_eval.shape, y_eval.shape
print("")
lgs = LogisticRegression(class_weight = 'balanced')
lgs.fit(X_train, y_train)
predicted = lgs.predict(X_eval)
print("Accuracy: %f" % lgs.score(X_eval, y_eval))

FileNotFoundError: ignored

In [54]:
from keras.models import Sequential
from keras.layers import Dense
X_train1 = X_train.toarray()
X_eval1 = X_eval.toarray()

model = Sequential()
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics = ['accuracy'])
model.fit(X_train1, y_train, epochs=200, verbose=1)

_, accuracy = model.evaluate(X_eval1, y_eval)
print('Accuracy: ', accuracy)
model.summary()

NameError: ignored